In [1]:
import pandas as pd
import numpy as np

# format sig figs
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Append Observed to Modeled (previously completed)
## Add the script here...

In [2]:
#############################
# Load the observed records
# 
obs = pd.read_csv(r'R:\FastTrips\CHTS and OBS ft_output\OBS_ft_output_w_stops.csv')
obs.fillna('',inplace=True)
obs['person_id'] = obs['person_id'].apply(lambda x: x.split("_")[-1]).astype('str')
obs['A_id_num'] = obs['A_id_num'].replace("",0)
obs['A_id_num'] = obs['A_id_num'].astype('int')
obs = obs.drop('trip_list_id_num',axis=1)

# ADD THE PROPER trip_list_id_num
obsinput = pd.read_csv(r'J:\Projects\FasTrips\obs\input\1.0\OBS_fasttrips_demand_v1.0\trip_list.txt')
obsinput['person_id'] = obsinput['person_id'].astype('int').astype('str')

obs = pd.merge(obs, obsinput[['person_id','trip_list_id_num']], on='person_id')
obs.to_csv('R:\FastTrips\CHTS and OBS ft_output\OBS_ft_output_w_stops_BN.csv')
# Make sure observed matches modeled


##########################
# On Board Survey Results
model = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\chosenpaths_links.csv')
model['route_id'].fillna("",inplace=True)
# FOR NOW, assume we look at only the last iteration of path sets ?
model = model[model['iteration'] == model['iteration'].max()]
model['person_id'] = model['person_id'].astype('int').astype('str')


#########################
# Pathsets
pathset_links = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\pathset_links.csv')
pathset_links['route_id'].fillna("",inplace=True)
pathset_links = pathset_links[pathset_links['iteration'] == pathset_links['iteration'].max()]

# For OBS
pathset_paths = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\pathset_paths.csv')


In [123]:
# pathset_paths = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\pathset_paths.csv')

In [3]:
# Load data CHTS

# # For testing, use a known list of trips, e.g., fast trips output
# obs = pd.read_csv(r'J:\Projects\FasTrips\chts\output\CHTS_fasttrips_demand_v0.1_stochastic_iter2_nocap\chosenpaths_links.csv')
# # Won't need to do this with real observed data
# obs['route_id'].fillna("",inplace=True)
# obs = obs[obs['iteration'] == obs['iteration'].max()]

# CHTS
# Load the model records
# model = pd.read_csv(r'J:\Projects\FasTrips\chts\output\CHTS_fasttrips_demand_v0.1_stochastic_iter2_nocap\chosenpaths_links.csv')
# model['route_id'].fillna("",inplace=True)
# # FOR NOW, assume we look at only the last iteration of path sets ?
# model = model[model['iteration'] == model['iteration'].max()]

# Load pathset links
## CHTS
# pathset_links = pd.read_csv(r'J:\Projects\FasTrips\tableau\path-set\pathset_links.csv')
# pathset_links['route_id'].fillna("",inplace=True)
# pathset_links = pathset_links[pathset_links['iteration'] == pathset_links['iteration'].max()]

# Reset Indeces

In [4]:
obs.reset_index(inplace=True)
model.reset_index(inplace=True)
pathset_links.reset_index(inplace=True)
pathset_paths.reset_index(inplace=True)

In [5]:
# 

# Create Unique ID

In [6]:
obs['unique_id'] = obs['person_id'].astype('str') +"_"+obs['trip_list_id_num'].astype('str')
model['unique_id'] = model['person_id'].astype('str') +"_"+model['trip_list_id_num'].astype('str')
pathset_links['unique_id'] = pathset_links['person_id'].astype('int').astype('str') +"_"+pathset_links['trip_list_id_num'].astype('str')
pathset_paths['unique_id'] = pathset_paths['person_id'].astype('str') +"_"+pathset_paths['trip_list_id_num'].astype('str')
pathset_paths['person_id'] = pathset_paths['person_id'].astype('int').astype('str')
pathset_paths['unique_id'] = pathset_paths['person_id'].astype('str') +"_"+ pathset_paths['trip_list_id_num'].astype('str')

** each unique ID should exist within the observed and the model results **

In [7]:
# Compare the unique_id fields between each data set
print len(obs.groupby('unique_id').count().index)
print len(model.groupby('unique_id').count().index)
print len(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))

22951
20836
18131


Some observed trips were filtered out, but the number of modeled trips should be the same as the 
number of common unique_ids. For some reason it's less

In [8]:
# Use the common unique_ids to filter out the data
model = model[model['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))]
obs = obs[obs['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))]

In [9]:
print len(obs.groupby('unique_id').count().index)
print len(model.groupby('unique_id').count().index)
print len(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))

18131
18131
18131


**Compare against the pathset links and paths files**

In [10]:
print len(obs.groupby('unique_id').count().index)
print len(pathset_links.groupby('unique_id').count().index)
print len(list(set((obs['unique_id'].values)).intersection((pathset_links['unique_id'].values))))
obs = obs[obs['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))]
pathset_links = pathset_links[pathset_links['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((model['unique_id'].values))))]

18131
25348
18131


In [11]:
print len(obs.groupby('unique_id').count().index)
print len(pathset_paths.groupby('unique_id').count().index)
print len(list(set((obs['unique_id'].values)).intersection((pathset_paths['unique_id'].values))))
obs = obs[obs['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((pathset_paths['unique_id'].values))))]
pathset_paths = pathset_paths[pathset_paths['unique_id'].isin(list(set((obs['unique_id'].values)).intersection((pathset_paths['unique_id'].values))))]

18131
25441
18131


# Join correct transit route IDs to OBS data

# Add a transit agency lookup too

In [12]:
def produce_path_fields(df, group):
    '''
    '''
    # create "path_routes"
    df['path_routes'] = df['route_id'].apply(lambda x: x.strip())
    path_routes = pd.DataFrame(df.groupby(group)['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    result_df = pd.DataFrame(index=path_routes.index)
    result_df['path_routes'] = path_routes
    
    # create "path_modes"
    df['path_modes'] = df['mode'].apply(lambda x: x.strip())
    result_df['path_modes'] = pd.DataFrame(df.groupby(group)['mode'].apply(lambda x: "%s" % ' '.join(x).strip()))

    # Create "path_components"
    df['path_components'] = df['A_id'].astype('str')+" "+df['mode']+" "+df['route_id'] +"_"+ df['B_id'].astype('str')
    df['path_components'] = df['path_components'].apply(lambda x: x.strip())
    result_df['path_components'] = pd.DataFrame(df.groupby(group)['path_components'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    # Return ID field from index
    result_df['unique_id'] = result_df.index.get_level_values(0).values
    

    return result_df

# Produce joined fields from pathset link files 

In [13]:
# Create caluclated joined fields for the observed, modeled, and pathset links file

# Concatenate modes, route IDs, etc to produce unique trip identities
# Do this for each set of trips in the oberved data, as well as for the modeled, since we don't have the field produced by FT
observed_path = produce_path_fields(obs, group=['unique_id'])

In [14]:
modeled_path = produce_path_fields(model, group=['unique_id'])

In [15]:
# also process the detailed pathset_links files, so each path in the pathset has a unique trip identity
new_pathset = produce_path_fields(pathset_links, group=['unique_id','pathnum'])

### Weirdness with unique_id mismatch between model/obs and the pathset_links file

In [16]:


################################
# For some reason there is not a smooth overlap between the modeled pathset_link unique_ids and the model/observed data
# there are unique IDs missing from each, so I guess we just take an intersection of what's available for all of them?
print len(new_pathset)
print len(model)
print len(obs)

305995
94335
70988


In [250]:
# pathset_links['unique_id'] == ''

In [18]:
obs = obs[obs['unique_id'].isin(new_pathset['unique_id'].values)]
model = model[model['unique_id'].isin(new_pathset['unique_id'].values)]
new_pathset = new_pathset[new_pathset['unique_id'].isin(obs['unique_id'].values)]
new_pathset = new_pathset[new_pathset['unique_id'].isin(model['unique_id'].values)]

In [19]:
print len(new_pathset)
print len(model)
print len(obs)

305995
94335
70988


## Compare if modeled/observed trips match, completed or partially

In [259]:
# Join the observed and modeled fields
df = pd.merge(observed_path, modeled_path, on='unique_id',suffixes=("_observed","_model"))

In [260]:
# Find rows with matching path routes
complete_route_match = df[df['path_routes_observed'] == df['path_routes_model']]
complete_mode_match = df[df['path_modes_observed'] == df['path_modes_model']]
# Add complete_agency_match when available
# complete_route_match = df[df['path_agency_observed'] == df['path_agency_model']]

In [261]:
## Extract order of transit routes taken
df['model_path_route_list'] = df['path_routes_model'].apply(lambda x: x.split(" "))
df['obs_path_route_list'] = df['path_routes_observed'].apply(lambda x: x.split(" "))

df['model_path_mode_list'] = df['path_modes_model'].apply(lambda x: x.split(" "))
df['obs_path_mode_list'] = df['path_modes_observed'].apply(lambda x: x.split(" "))

In [262]:
# Isolate transit modes only, because almost all trips have walk & transfer components
non_transit_modes = ['transfer','walk_access','walk_egress','bike_access','bike_egress',
                     'PNR_access','PNR_egress','KNR_access','KNR_egress']
df['model_transit_modes'] = df['model_path_mode_list'].apply(
    lambda row: [element for element in row if element not in non_transit_modes])
df['obs_transit_modes'] = df['obs_path_mode_list'].apply(
    lambda row: [element for element in row if element not in non_transit_modes])

In [263]:

# Find the intersection between the chosen model/observed paths using different criteria

# transit route IDs only
df.apply(lambda row: all(i in row['model_path_route_list'] for i in row['obs_path_route_list']), axis=1)
df['routes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_route_list'], df['obs_path_route_list'])]

# All Modes (including transfer, access/egress)
df.apply(lambda row: all(i in row['model_path_mode_list'] for i in row['obs_path_mode_list']), axis=1)
df['all_modes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_mode_list'], df['obs_path_mode_list'])]

# Transit modes only (type of vehicle taken and number of boardings)
df.apply(lambda row: all(i in row['model_path_mode_list'] for i in row['obs_path_mode_list']), axis=1)
df['transit_modes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_mode_list'], df['obs_path_mode_list'])]

# Next steps: 
# Agency Intersection

# All path components

# Number of boardings?

# Exact Match of Path Routes, Modes, Components

In [264]:
# Exact match of path modes
complete_mode_match['complete_mode_match'] = 1
df = pd.merge(df, complete_mode_match[['unique_id','complete_mode_match']], how='left', on='unique_id')
df['complete_mode_match'].fillna(0,inplace=True)

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [265]:
complete_route_match['complete_route_match'] = 1

In [266]:
df = pd.merge(df, complete_route_match[['unique_id','complete_route_match']], how='left', on='unique_id')

In [267]:
df['complete_route_match'] = df['complete_route_match'].replace('nan',0)


In [268]:
df['complete_route_match'].mean()

0.0

In [269]:
df['complete_mode_match'].mean()

0.2227124813854724

## Export fields to CSV for Tableau:
- 'complete_route_match' 0/1
- 'complete_mode_match' 0/1

In [274]:
len(df)

18131

# % trips with matching or partial matching routes

In [31]:
# Now we find the percent of trips with matching routes or partial matching routes
# df.mean()[['complete_match','partial_match']]
# Note, shuold be 100% when using the same data set, what's up??

#############
# Join the filtered data to the original results
df['common_mode_count'] = [len(row) for row in df['all_modes_intersection']]
df['common_transit_mode_count'] = [len(row) for row in df['transit_modes_intersection']]

# How many rows have at least one mode in common?
df['partial_mode_match'] = [1 if row > 0 else 0 for row in df['common_mode_count']]
df['partial_transit_mode_match'] = [1 if row > 0 else 0 for row in df['common_transit_mode_count']]

In [32]:

df['partial_mode_match'].mean()

0.9986763002592245

In [33]:
df['partial_transit_mode_match'].mean()

0.9986763002592245

## Export fields to CSV for Tableau:
- 'partial_mode_match' 0/1
- 'complete_mode_match' 0/1

# Compare probability of observed path to pathset
# & Check if path is in pathset


In [68]:
observed_path.head(3)

,path_routes,path_modes,path_components,unique_id
unique_id,,,,
100066_15335,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335
1000_431,MUNI Caltrain_San Francisco Burlingame Tri-...,walk_access local_bus transfer commuter_rail w...,930.0 walk_access _24893.0 24893.0 local_bus M...,1000_431
1000_432,MUNI Caltrain_San Francisco Burlingame Tri-...,walk_access local_bus transfer commuter_rail w...,930.0 walk_access _24893.0 24893.0 local_bus M...,1000_432


In [110]:
new_pathset.head(3)

path_routes                         path_modes  \
unique_id    pathnum                                                  
100066_15335 0          76_14MNO  walk_access local_bus walk_egress   
             1          74_44NBL  walk_access local_bus walk_egress   
             2          74_62NBL  walk_access local_bus walk_egress   

                                                        path_components  \
unique_id    pathnum                                                      
100066_15335 0        2361 walk_access _9376 9376 local_bus 76_14MNO...   
             1        2361 walk_access _9376 9376 local_bus 74_44NBL...   
             2        2361 walk_access _9376 9376 local_bus 74_62NBL...   

                         unique_id  pathnum  
unique_id    pathnum                         
100066_15335 0        100066_15335        0  
             1        100066_15335        1  
             2        100066_15335        2

In [78]:
## Add a field to the new_pathset that lists the pathnum
new_pathset['pathnum'] = new_pathset.index.get_level_values(1)

In [79]:
# Do this with a merge?

df = pd.merge(observed_path, new_pathset, how='left',
              left_on=['unique_id','path_modes'],right_on=['unique_id','path_modes'], suffixes=['_obs','_pathset'])

In [80]:
print len(df)
print len(observed_path)
print len(new_pathset)

46866
18131
305995


In [98]:
# How many unique ID's don't have a pathset attacheds
len(df[df['pathnum'].isnull()])/float(len(df))

0.28141936585157684

In [107]:
df.head(3)

,path_routes_obs,path_modes,path_components_obs,unique_id,path_routes_pathset,path_components_pathset,pathnum
0,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,76_14MNO,2361 walk_access _9376 9376 local_bus 76_14MNO...,0.00000
1,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,74_44NBL,2361 walk_access _9376 9376 local_bus 74_44NBL...,1.00000
2,Sonoma County_62,walk_access local_bus walk_egress,2361.0 walk_access _ local_bus Sonoma County_6...,100066_15335,74_62NBL,2361 walk_access _9376 9376 local_bus 74_62NBL...,2.00000


In [116]:
df['pathnum'] = df['pathnum'].fillna(0)
df['pathnum'] = df['pathnum'].astype('int')

In [131]:
# Do we already have a unique ID? Try it anyway
pathset_paths['unique_id'] = pathset_paths['person_id'].astype('int').astype('str')+"_"+pathset_paths['trip_list_id_num'].astype('int').astype('str')

In [138]:
# Now look up the probability of each path between new_pathset and pathset_paths
newdf = pd.merge(df,pathset_paths,left_on=['unique_id','pathnum'], right_on=['unique_id','pathnum'])

In [140]:
newdf['probability'] = newdf['probability'].fillna('no_match')

In [145]:
max_prob = newdf.groupby('unique_id').max()['probability']

# to take the mean, need to do some filtering
# mean_prob = newdf.groupby('unique_id').()['probability']
min_prob = newdf.groupby('unique_id').min()['probability']

In [229]:
#Create indicator for paths that exist
# New dataframe that has prob matching record for each unique ID
prob_export = pd.DataFrame([max_prob,min_prob]).T
prob_export.columns = ['max_prob','min_prob']

In [230]:
# Pull binary data for each person
prob_export['path_exists'] = prob_export['max_prob'].apply(lambda row_value: 0 if row_value == 'no_match' else 1)

In [231]:
prob_export['path_exists'].mean()

0.7417682422370525

In [181]:
# Path exists in the pathset, based on mode strings, for 25% of all trips

In [240]:
# Is the max probability above a defined threshold?
threshold = 0.3
# Mark no_match_records
prob_export.ix[prob_export['max_prob'] >= threshold, 'above_threshold'] = 1
prob_export.ix[prob_export['max_prob'] < threshold, 'above_threshold'] = 0
prob_export.ix[prob_export['max_prob'] == 'no_match', 'above_threshold'] = 'no_match'

In [246]:
# Percent of trips above a threshold
prob_export[prob_export['above_threshold'] != "no_match"].mean()

max_prob          0.48796
min_prob          0.29901
path_exists       1.00000
above_threshold   0.63953
dtype: float64

In [247]:
###
# 63% of observed paths have a max pathest probability greather than a threshold of 0.3


## Export the trip-based (unique_id) records as a new input for Tableau

# Join all relevant columns & export to csv

In [278]:
prob_export['unique_id'] = prob_export.index

In [280]:
export_df = pd.merge(df, prob_export, on='unique_id')

In [282]:
len(export_df) == len(df) == len(prob_export)

True

In [284]:
# We can consider dropping columns if needed
export_df.columns

Index([u'path_routes_observed', u'path_modes_observed', u'path_components_observed', u'unique_id', u'path_routes_model', u'path_modes_model', u'path_components_model', u'model_path_route_list', u'obs_path_route_list', u'model_path_mode_list', u'obs_path_mode_list', u'model_transit_modes', u'obs_transit_modes', u'routes_intersection', u'all_modes_intersection', u'transit_modes_intersection', u'complete_mode_match', u'complete_route_match', u'max_prob', u'min_prob', u'path_exists', u'above_threshold'], dtype='object')

In [285]:
export_df.to_csv('path_comparison.csv')